In [22]:
import torch
from einops import rearrange, einsum


In [23]:
D = torch.randn(5,2,3)
A = torch.randn(2,3)
D @ A.T

tensor([[[-1.7128,  0.3073],
         [ 0.0575,  1.3462]],

        [[-0.7967,  0.0265],
         [ 0.5132,  1.6646]],

        [[-0.8223,  0.3482],
         [-0.7530,  0.8034]],

        [[ 0.4352, -0.3029],
         [ 2.5822, -2.9994]],

        [[-0.5144,  0.3487],
         [ 0.4417, -0.6248]]])

In [24]:
# 假设使用 einops.einsum
# 修正后的 pattern：
Y = einsum(D, A, "batch sequence d_in, d_out d_in -> batch sequence d_out")
Y

tensor([[[-1.7128,  0.3073],
         [ 0.0575,  1.3462]],

        [[-0.7967,  0.0265],
         [ 0.5132,  1.6646]],

        [[-0.8223,  0.3482],
         [-0.7530,  0.8034]],

        [[ 0.4352, -0.3029],
         [ 2.5822, -2.9994]],

        [[-0.5144,  0.3487],
         [ 0.4417, -0.6248]]])

In [25]:
Y = einsum(D,A,"... d_in, dout d_in -> ... dout")
Y

tensor([[[-1.7128,  0.3073],
         [ 0.0575,  1.3462]],

        [[-0.7967,  0.0265],
         [ 0.5132,  1.6646]],

        [[-0.8223,  0.3482],
         [-0.7530,  0.8034]],

        [[ 0.4352, -0.3029],
         [ 2.5822, -2.9994]],

        [[-0.5144,  0.3487],
         [ 0.4417, -0.6248]]])

In [26]:
images = torch.randn(64,128,128,3)
dim_by = torch.linspace(start=0,end=1.0,steps=10)
dim_by.shape

torch.Size([10])

In [27]:
dim_value = rearrange(dim_by, "dim_value -> 1 dim_value 1 1 1")
images_rearr = rearrange(images, "b height width channel -> b 1 height width channel")
dimmed_images = images_rearr * dim_value
dimmed_images.shape

torch.Size([64, 10, 128, 128, 3])

In [29]:
dimmed_images = einsum(images,dim_by,"batch height width channel, dim_value -> batch dim_value height width channel")
dimmed_images.shape

torch.Size([64, 10, 128, 128, 3])

In [30]:
channels_last = torch.randn(64,32,32,3)
B = torch.randn(32*32, 32*32)

In [35]:
channels_last_flat = channels_last.view(-1,channels_last.size(1) * channels_last.size(2),channels_last.size(3))
channels_first_flat = channels_last_flat.transpose(1,2)
channels_first_flat.shape

torch.Size([64, 3, 1024])

In [39]:
channels_first_flat_transformed = channels_first_flat @ B.T
channels_last_flat_transformed = channels_first_flat_transformed.transpose(1,2)
channels_last_flat_transformed = channels_last_flat_transformed.view(*channels_last.shape)



In [46]:
height = width = 32
channels_first = rearrange(channels_last,"batch height width channel->batch channel (height width)")
channels_first_transformed = einsum(channels_first,B,"batch channel pixel_in, pixel_out pixel_in -> batch channel pixel_out")
channels_last_transformed = rearrange(channels_first_transformed,"batch channel (height width)->batch height width channel",height = height,width = width)
